In [57]:
import pandas as pd
import sqlite3
from pybliometrics.scopus import AbstractRetrieval
from six import string_types, text_type
from crossref.restful import Works
# import ibis
import sys
sys.path.append("..")
from rcn_py import orcid


In [3]:
df_peter = pd.read_csv("/Users/jennifer/RCSNA/data/Peter Kalverla_crossref.csv")

In [10]:
con = sqlite3.connect("tutorial.db")

In [11]:
cur = con.cursor()


In [12]:
cur.execute("CREATE TABLE test_paper(doi primary key, title, abstract)")

In [13]:
cur.execute("CREATE TABLE test_authors(orcid primary key, name, country)")


In [16]:
cur.execute("CREATE TABLE test_author_papers2(orcid, doi, primary key(orcid, doi) FOREIGN KEY(orcid) REFERENCES test_author(orcid), FOREIGN KEY(doi) REFERENCES test_paper(doi))")

In [55]:
con.commit()

In [56]:
con.close()

In [76]:
''' 
    author-paper table
    author info table
    publication table
'''

def insert_database(orcid_id, fullname): 
    if len(orcid_id)==0:
        orcid_id = orcid.name_to_orcid_id(fullname)
    orcid_record = orcid.query_orcid_for_record(orcid_id)
    # probably there is null value in every stage, which will occure an error
    if len(orcid_record['person']['addresses']['address']) > 0:
        country = orcid_record['person']['addresses']['address'][0]['country']['value']
    else:
        country = ''

    docs = orcid.extract_works_section(orcid_record)
    
    con = sqlite3.connect("tutorial.db")
    cur = con.cursor()
    works = Works()
    orcid_doi_value = []
    paper_value = []
    for doc in docs:
        doi, title = orcid.extract_doi(doc)
        # make insert value for orcid-doi
        orcid_doi_value.append((orcid_id, doi))

        w = works.doi(doi)
        # orcid_list, name_list = orcid.get_authors(doi)
        
        if w:
            if 'abstract' in w.keys():
                abstract = w['abstract']
            else:
                abstract = ''
        else:
            abstract = ''

        paper_value.append((doi, title, abstract))
        
    # author table (orcid, name, country)
    cur.execute("INSERT or IGNORE INTO authors VALUES (?, ?, ?)", (orcid_id, fullname, country))
    # publication table (doi, title, abstract)
    cur.executemany("INSERT or IGNORE INTO publications VALUES (?, ?, ?)", paper_value)
    # author-paper table (orcid, doi)
    cur.executemany("INSERT or IGNORE INTO author_publication VALUES (?, ?)", orcid_doi_value)
    con.commit()
    con.close()

    return "Done"
    

In [72]:
insert_database('',"Peter Kalverla")

'Done.'

In [83]:
# Retrieve the coauthors for publications of the person
def insert_coauthors_data(fullname):
    orcid_id = orcid.name_to_orcid_id(fullname)
    orcid_record = orcid.query_orcid_for_record(orcid_id)
    if orcid_record == False:
        return
    docs = orcid.extract_works_section(orcid_record)

    for doc in docs:
        doi, title = orcid.extract_doi(doc)
        orcid_list, name_list = orcid.get_authors(doi)
        for i in range(len(orcid_list)): 
            insert_database(orcid_list[i], name_list[i])

    return "Done!"
    

In [84]:
insert_coauthors_data("Peter Kalverla")

TypeError: 'NoneType' object is not subscriptable

In [88]:
works = orcid_record['activities-summary']['works']['group']

In [89]:
works[0]

{'last-modified-date': {'value': 1654040356185},
 'external-ids': {'external-id': [{'external-id-type': 'doi',
    'external-id-value': '10.5194/egusphere-egu21-9387',
    'external-id-normalized': {'value': '10.5194/egusphere-egu21-9387',
     'transient': True},
    'external-id-normalized-error': None,
    'external-id-url': {'value': 'https://doi.org/10.5194/egusphere-egu21-9387'},
    'external-id-relationship': 'self'}]},
 'work-summary': [{'put-code': 89903286,
   'created-date': {'value': 1614838570891},
   'last-modified-date': {'value': 1654040356185},
   'source': {'source-orcid': None,
    'source-client-id': {'uri': 'https://orcid.org/client/0000-0001-9884-1913',
     'path': '0000-0001-9884-1913',
     'host': 'orcid.org'},
    'source-name': {'value': 'Crossref'},
    'assertion-origin-orcid': None,
    'assertion-origin-client-id': None,
    'assertion-origin-name': None},
   'title': {'title': {'value': 'A multi-model ensemble weighting method (ClimWIP) in ESMValTool'}

In [90]:
work_summary = works[0]['work-summary'][0]

In [91]:
work_summary

{'put-code': 89903286,
 'created-date': {'value': 1614838570891},
 'last-modified-date': {'value': 1654040356185},
 'source': {'source-orcid': None,
  'source-client-id': {'uri': 'https://orcid.org/client/0000-0001-9884-1913',
   'path': '0000-0001-9884-1913',
   'host': 'orcid.org'},
  'source-name': {'value': 'Crossref'},
  'assertion-origin-orcid': None,
  'assertion-origin-client-id': None,
  'assertion-origin-name': None},
 'title': {'title': {'value': 'A multi-model ensemble weighting method (ClimWIP) in ESMValTool'},
  'subtitle': None,
  'translated-title': None},
 'external-ids': {'external-id': [{'external-id-type': 'doi',
    'external-id-value': '10.5194/egusphere-egu21-9387',
    'external-id-normalized': {'value': '10.5194/egusphere-egu21-9387',
     'transient': True},
    'external-id-normalized-error': None,
    'external-id-url': {'value': 'https://doi.org/10.5194/egusphere-egu21-9387'},
    'external-id-relationship': 'self'}]},
 'url': {'value': 'https://doi.org/10.

In [92]:
dois =  [doi['external-id-value'] for doi in work_summary['external-ids']['external-id'] if doi if doi['external-id-type']=="doi"]
 

In [93]:
dois

['10.5194/egusphere-egu21-9387']

In [95]:
dois = []
if 'external-id' in work_summary['external-ids'].keys():
    for ws in work_summary['external-ids']['external-id']:
        if 'external-id-type' in ws.keys() and 'external-id-value' in ws.keys():
            if ws['external-id-type']=="doi":
                dois.append(ws['external-id-value'])

In [97]:
dois[0]

'10.5194/egusphere-egu21-9387'